### IMPORTING THE LIBRARIES

In [ ]:
# modules
import noise

# libraries
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np 
import sys
from pathlib import Path
from PIL import Image
import string
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### LOADING THE IMAGE TO BE PROCESSED

In [ ]:
roseImg = Image.open('rose.png')
plt.imshow(roseImg)
plt.subplot()

In [ ]:
car = Image.open('car.jpg')

In [ ]:
house = Image.open('house.jpg')

In [ ]:
balloons = Image.open('balloons.png')

In [ ]:
roseImg.size

### Transformation on Image

#### Affine Transformation

In [ ]:
A = np.array([[0,1,0], [1,0,1],[0,-1,2],[ -1,0,3]])

In [ ]:
#The identity matrix with the same size as A
I = np.eye(3)

In [ ]:
rgb_color_palette = 'rgbc'

In [ ]:
fig = plt.figure()
ax = plt.gca()
x_cordinates = []
y_cordinates = []

for row in A:
    output_row = I @ row
    x, y, i = output_row
    x_cordinates.append(x)
    y_cordinates.append(y)
    c = rgb_color_palette[int(i)]
    plt.scatter(x, y, color=c)
    plt.text(x + 0.15, y, f"{string.ascii_letters[int(i)]}")
x_cordinates.append(x_cordinates[0])
y_cordinates.append(y_cordinates[0])
plt.plot(x_cordinates, y_cordinates, color="orange")
# ax.set_xticks(np.arange(-2.5, 3, 0.5))
# ax.set_yticks(np.arange(-2.5, 3, 0.5))
plt.grid()
plt.show()

#### scaling the object using a transformation matrix

In [ ]:
Trans_matrix= np.array([[2, 0, 0], [0, 2, 0], [0, 0, 1]])   #transformation matrix

fig = plt.figure()
ax = plt.gca()
x_cors = []
y_cors = []
for row in A:
    output_row = Trans_matrix @ row
    x, y, i = row
    x_units, y_units, indx = output_row
    print(x_units)
    x_cors.append(x_units)
    y_cors.append(y_units)
    c, c_s = rgb_color_palette[int(i)], rgb_color_palette[int(indx)]
    plt.scatter(x, y, color=c)
    plt.scatter(x_units, y_units, color=c_s)
    plt.text(x + 0.15, y, f"{string.ascii_letters[int(i)]}")
    plt.text(x_units + 0.15, y_units, f"{string.ascii_letters[int(indx)]}'")

x_cors.append(x_cors[0])
y_cors.append(y_cors[0])
plt.plot(x_cordinates, y_cordinates, color="orange", linestyle = 'solid')
plt.plot(x_cors, y_cors, color="violet", linestyle='dotted')
ax.set_xticks(np.arange(-2.5, 3, 0.5))
ax.set_yticks(np.arange(-2.5, 3, 0.5))
plt.grid()
plt.show()

#### rotation through 90 degrees using a transformation matrix

In [ ]:
Trans_matrix_rot= np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 1]])

fig = plt.figure()
ax = plt.gca()
for row in A:
    output_row = Trans_matrix_rot @ row 
    x_pts, y_pts, indx_ = output_row
    c_r = rgb_color_palette[indx_] 
    letter_label = string.ascii_letters[indx_]
    plt.scatter(x_pts, y_pts, color=c_r)
    plt.text(x_pts + 0.15, y_pts, f"{letter_label}'")

plt.plot(x_cordinates, y_cordinates, color="orange", linestyle = 'solid')
ax.set_xticks(np.arange(-2.5, 3, 0.5))
ax.set_yticks(np.arange(-2.5, 3, 0.5))
plt.grid()
plt.show()

#### Now, transforming the rose image using the idea above

In [ ]:
# recenter resultant image
T_center = np.array([
    [1, 0, 1000],
    [0, 1, 1000],
    [0, 0, 1]])
# rotate - opposite angle
T_rotate = np.array([
    [0, -1, 0],
    [1, 0, 0],
    [0, 0, 1]])
# scale
T_scale = np.array([
    [2, 0, 0],
    [0, 2, 0],
    [0, 0, 1]])
# center original to 0,0
T_neg500 = np.array([
    [1, 0, -200],
    [0, 1, -200],
    [0, 0, 1]])
Transformed_img = T_center @ T_rotate @ T_scale @ T_neg500
Trans_img_inv = np.linalg.inv(T)

In [ ]:
T_inv

In [ ]:
img_transformed = roseImg.transform((2000, 2000), Image.AFFINE, data=T_inv.flatten()[:6], resample=Image.NEAREST)
plt.imshow(np.asarray(img_transformed))

#### Reiszing the roseImg using mulptiplication by scalar 

In [ ]:
input_size = 1500
output_size = 750
bin_size = input_size // output_size
small_image = roseImg.resize((output_size,bin_size))



#### Translation on Image

### DENOISING AN IMAGE

In [ ]:
def get_image(filename):
    # user_input = input("Enter directory: ")
    image_data = img.imread(filename)
    return image_data


In [ ]:
def find_new_s(s_row, s_column, s):
    new_s = np.zeros((s_row, s_column))
    for i in range(0, s.shape[0], 5):
        if s[i] > 1750:
            new_s[i][i] = s[i]
    return new_s


In [ ]:
def denoise_image(noisy_image):
    """
    The above function takes a noisy image as input and returns a cleaned image.
    :param noisy_image: the noisy image
    """
    row = noisy_image.shape[0]
    column = noisy_image.shape[1]
    matrix0 = np.zeros((row, column))
    for r in range(0, row, 1):
        for c in range(0, column, 1):
            matrix0[r][c] = noisy_image[r][c][0]
    u0, s0, v0 = np.linalg.svd(matrix0)
    s_row = u0.shape[1]
    s_column = v0.shape[0]
    new_s0 = find_new_s(s_row, s_column, s0)

    matrix1 = np.zeros((row, column))
    for r in range(0, row, 1):
        for c in range(0, column, 1):
            matrix1[r][c] = noisy_image[r][c][1]
    u1, s1, v1 = np.linalg.svd(matrix1)
    new_s1 = find_new_s(s_row, s_column, s1)

    matrix2 = np.zeros((row, column))
    for r in range(0, row, 1):
        for c in range(0, column, 1):
            matrix2[r][c] = noisy_image[r][c][2]
    u2, s2, v2 = np.linalg.svd(matrix2)
    new_s2 = find_new_s(s_row, s_column, s0)

    new_image = np.zeros((row, column, 3), dtype='uint8')
    matrix0 = np.dot(u0, new_s0)
    matrix0 = np.dot(matrix0, v0)
    matrix1 = np.dot(u1, new_s1)
    matrix1 = np.dot(matrix1, v1)
    matrix2 = np.dot(u2, new_s2)
    matrix2 = np.dot(matrix2, v2)
    for r in range(0, row, 1):
        for c in range(0, column, 1):
            new_image[r][c][0] = matrix0[r][c]
            new_image[r][c][1] = matrix1[r][c]
            new_image[r][c][2] = matrix2[r][c]
    plt.imsave('cleaned image.jpeg', new_image)
    plt.imshow(new_image)
    plt.show()


In [ ]:
image = get_image("balloons.png")

In [ ]:
denoise_image(image)

### Image Edge Detection

In [ ]:

vertical_filter = [[-1,-2,-1], [0,0,0], [1,2,1]]
horizontal_filter = [[-1,0,1], [-2,0,2], [-1,0,1]]
img = plt.imread('greyscale.png')
n,m,d = img.shape

edges_img = img.copy()

for row in range(3, n-2):
    for col in range(3, m-2):
        local_pixels = img[row-1:row+2, col-1:col+2, 0]
        vertical_transformed_pixels = vertical_filter*local_pixels
        vertical_score = vertical_transformed_pixels.sum()/4
        horizontal_transformed_pixels = horizontal_filter*local_pixels
        horizontal_score = horizontal_transformed_pixels.sum()/4
        edge_score = (vertical_score**2 + horizontal_score**2)**.5
        edges_img[row, col] = [edge_score]*3
edges_img = edges_img/edges_img.max()


#### Image Shawdowing

In [ ]:
    
def make_new_image(im):
    main_image = im
    shape = main_image.shape
    row_value = shape[0]
    column_value = shape[1] + 100
    new_image = np.zeros((row_value, column_value, 3), dtype='uint8')
    for row in range(0, shape[0], 1):
        for column in range(0, shape[1], 1):
            for RGB in range(0, 3, 1):
                if main_image[row][column][0] == 255 and main_image[row][column][1] == 255 and main_image[row][column][2] == 255:
                    new_image[row][column][RGB] = main_image[row][column][RGB]
                else:
                    new_image[row][column][RGB] = 100

    for row in range(0, shape[0], 1):
        for column in range(shape[1], column_value, 1):
            for RGB in range(0, 3, 1):
                new_image[row][column][RGB] = 255
    plt.figure(figsize=(15,15))
    plt.imshow(new_image)
    plt.show()
    return new_image


In [ ]:
def shear_transformation(img):
    gray_image = img
    shape = gray_image.shape
    row_value = shape[0]
    column_value = shape[1]
    new_image = np.zeros((row_value, column_value+gray_image.shape[0], 3), dtype='uint8')
    for row in range(0, shape[0], 1):
        for column in range(0, shape[1], 1):
            for RGB in range(0, 3, 1):
                new_image[row][-row+column][RGB] = gray_image[row][column][RGB]

    for row in range(0, new_image.shape[0], 1):
        for column in range(0, new_image.shape[1], 1):
            if new_image[row][column][0] == 0 and new_image[row][column][1] == 0 and new_image[row][column][2] == 0:
                for RGB in range(0, 3, 1):
                    new_image[row][column][RGB] = 255

    correct_image = np.zeros((new_image.shape[0], new_image.shape[1], 3), dtype='uint8')
    for row in range(0, correct_image.shape[0], 1):
        for column in range(0, int(correct_image.shape[1] / 2), 1):
            for RGB in range(0, 3, 1):
                correct_image[row][int(correct_image.shape[1] / 2) + column][RGB] = new_image[row][column][RGB]

    for row in range(0, correct_image.shape[0], 1):
        for column in range(int(correct_image.shape[1]/2)+1, correct_image.shape[1], 1):
            for RGB in range(0, 3, 1):
                correct_image[row][int(-correct_image.shape[1]/2)+column][RGB] = new_image[row][column][RGB]
    plt.figure(figsize=(15, 15))
    plt.imshow(correct_image)
    plt.show()
    return correct_image


In [ ]:

def make_final_image(f_img, s_image):
    shape_first = f_img.shape
    shape_sheared = s_image.shape
    final_img = np.zeros((shape_sheared[0], shape_sheared[1], 3), dtype='uint8')
    for row in range(0, shape_first[0], 1):
        for column in range(0, shape_first[1], 1):
            if f_img[row][column][0] == 255 and f_img[row][column][1] == 255 and f_img[row][column][2] == 255:
                for RGB in range(0, 3, 1):
                    final_img[row][column][RGB] = s_image[row][column][RGB]

    for row in range(0, shape_sheared[0], 1):
        for column in range(shape_first[1], shape_sheared[1], 1):
            for RGB in range(0, 3, 1):
                final_img[row][column][RGB] = s_image[row][column][RGB]

    plt.figure(figsize=(15,15))
    plt.imshow(final_img)
    plt.show()

    for row in range(0, shape_first[0], 1):
        for column in range(0, shape_first[1], 1):
            if final_img[row][column][0] == 0 and final_img[row][column][1] == 0 and final_img[row][column][2] == 0:
                for RGB in range(0, 3, 1):
                    final_img[row][column][RGB] = f_img[row][column][RGB]

    plt.imshow(final_img)
    plt.show()


In [ ]:
new_ima = get_image('rose.jpg')
bird_image = make_new_image(new_ima)
sheared_image = shear_transformation(bird_image)
make_final_image(bird_image, sheared_image)

In [ ]:
PIL_image = Image.fromarray(np.uint8(edges_img))
PIL_image

  # PIL_image = Image.fromarray(edges_img.astype('uint8'), 'RGB')

### Image Compression